In [1]:
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy.stats as stats



In [20]:
featureList = ["BIRTHDAY","Sex","Race","Ethnicity","PreviouslyHealthy","PostOp","PRISMLowSBP","PRISMHighHeartRate",
              "PRISMLowTemp","PRISMHighTemp","GCS","PRISMIntubate","LowerPlatelet","LowPlatelet","HigherCreatinine",
              "LowLymphocyte","LowHemoglobin","OFICardio","OFIPulm","OFIHepatic","OFIRenal","OFIHemat","OFICNS","OFI",
              "SIRS","CRPH","Ferritin"];

In [29]:
# causalMGM_list.extend(deeppink_list);
# mergedList = set(causalMGM_list);
# print(len(mergedList))
deeppink_list = [];
causalMGM_list = [];
featureList = [];

In [30]:
nonlinearAssociations = pd.read_csv('Supplementary Data Nonlinear.csv',names=['F1','F2']);

for index,row in nonlinearAssociations.iterrows():
    f1 = row["F1"];
    f2 = row["F2"];
    tempAssos = [f1,f2];
    tempAssos.sort();
    tempStr = tempAssos[0]+"_"+tempAssos[1];
    deeppink_list.append(tempStr);
    if f1 not in featureList:
        featureList.append(f1);
    if f2 not in featureList:
        featureList.append(f2);

In [31]:
linearAssociations = pd.read_csv('Supplementary Data Linear.csv',names=['F1','F2']);
for index,row in linearAssociations.iterrows():
    f1 = row["F1"];
    f2 = row["F2"];
    tempAssos = [f1,f2];
    tempAssos.sort();
    tempStr = tempAssos[0]+"_"+tempAssos[1];
    causalMGM_list.append(tempStr);
    
    if f1 not in featureList:
        featureList.append(f1);
    if f2 not in featureList:
        featureList.append(f2);

In [32]:
# deeppink_list
print(len(set(deeppink_list)));
print(len(set(causalMGM_list)));
print(len(set(featureList)));

76
41
45


In [33]:
featureList

['HigherCreatinine',
 'PRISMLowSBP',
 'PRISMHighHeartRate',
 'IFN.a2',
 'PRISMLowTemp',
 'PRISMHighTemp',
 'LowerPlatelet',
 'IFN.b',
 'SIRS',
 'GCS',
 'CRPH',
 'IL.22',
 'TNF.a.VS',
 'SCF',
 'IL.1beta',
 'IFN.gamma',
 'IL.1a',
 'LowLymphocyte',
 'LowPlatelet',
 'IL.4',
 'MIG.CXCL9',
 'LowHemoglobin',
 'TRAIL',
 'ADAMTS.13',
 'IL.2Ra',
 'MCP.1.CCL2',
 'MIP.1a',
 'IL.18BP',
 'MCP.3',
 'sFas.Ligand',
 'IL.18',
 'IL.3',
 'IL.13',
 'TNF.a.SC',
 'M.CSF',
 'IL.17A',
 'IL.10',
 'BIRTHDAY',
 'sCD163',
 'MIP.1B',
 'Ferritin',
 'IL.16',
 'IP.10.CXCL10',
 'IL.6',
 'IL.8']

In [34]:
# print(len(set(deeppink_list).intersection(causalMGM_list)));
# print(len(set(deeppink_list).intersection(prior_list)));
# print(len(set(deeppink_list).intersection(causalMGM_list).intersection(prior_list)));
# print(len(set(causalMGM_list).intersection(prior_list)));

In [35]:
node_info = {}

print(len(deeppink_list));
print(len(causalMGM_list));

for ele in deeppink_list:
    fs = ele.split("_");
    if fs[0] not in node_info: 
        node_info[fs[0]] = {"DeepPINK":[fs[1]],"CausalMGM":[]};
    else:
        node_info[fs[0]]["DeepPINK"].append(fs[1]);
        
    if fs[1] not in node_info:
        node_info[fs[1]] = {"DeepPINK":[fs[0]],"CausalMGM":[]};
    else:
        node_info[fs[1]]["DeepPINK"].append(fs[0]);

for ele in causalMGM_list:
    fs = ele.split("_");
    if fs[0] not in node_info: 
        node_info[fs[0]] = {"DeepPINK":[],"CausalMGM":[fs[1]]};
    else:
        node_info[fs[0]]["CausalMGM"].append(fs[1]);
        
    if fs[1] not in node_info:
        node_info[fs[1]] = {"DeepPINK":[],"CausalMGM":[fs[0]]};
    else:
        node_info[fs[1]]["CausalMGM"].append(fs[0]);

76
42


In [36]:
node_info_sort = [];
for key,value in node_info.items():
    
    overlap = set(value["DeepPINK"]).intersection(value["CausalMGM"]);
    deepPINK_uniq = set(value["DeepPINK"])-overlap;
    causalMGM_uniq = set(value["CausalMGM"])-overlap;
    
    d_len = len(value["DeepPINK"]);
    c_len = len(value["CausalMGM"]);
    
    d_u_len = len(deepPINK_uniq);
    c_u_len = len(causalMGM_uniq);
    i_len = len(overlap);
    
    print(d_len);
    print(c_len);
    print(d_u_len);
    print(c_u_len);
    print(i_len);
    
    if d_len ==0:
        node_info_sort.append({"Feature":key,"D_P":0,"C_P":c_len,"I_P":0,"Dif":(d_len-c_len)});
    elif c_len == 0:
        node_info_sort.append({"Feature":key,"D_P":d_len,"C_P":0,"I_P":0,"Dif":(d_len-c_len)});
    else:
        sum = d_len+c_len;
        d_p = d_len/sum;
        c_p = c_len/sum;
        node_info_sort.append({"Feature":key,"D_P":d_u_len,"C_P":c_u_len,"I_P":i_len,"Dif":(d_p-c_p)});
print(len(node_info));


5
2
5
2
0
1
0
1
0
0
8
0
8
0
0
7
1
7
1
0
3
1
3
1
0
1
0
1
0
0
4
0
4
0
0
6
0
6
0
0
10
0
10
0
0
6
0
6
0
0
4
1
4
1
0
6
1
6
1
0
1
3
1
3
0
4
5
4
5
0
7
2
7
1
0
13
1
13
1
0
8
1
8
1
0
3
2
3
2
0
5
0
5
0
0
10
0
10
0
0
2
0
2
0
0
5
1
5
1
0
2
1
2
1
0
3
0
3
0
0
1
3
1
3
0
1
8
1
8
0
4
2
4
2
0
2
0
2
0
0
2
1
2
1
0
3
1
3
1
0
2
0
2
0
0
2
0
2
0
0
3
0
3
0
0
3
4
3
4
0
1
4
1
4
0
3
0
3
0
0
1
5
1
5
0
0
6
0
6
0
0
8
0
7
0
0
3
0
3
0
0
3
0
3
0
0
4
0
4
0
0
4
0
4
0
0
4
0
4
0
0
2
0
2
0
45


In [37]:
node_info_sort = pd.DataFrame(data=node_info_sort);
node_info_sort = node_info_sort.sort_values(by=['Dif']);
node_info_sort

,Feature,D_P,C_P,I_P,Dif
38,sCD163,0,8,0,-8.000000
37,BIRTHDAY,0,6,0,-6.000000
42,IP.10.CXCL10,0,4,0,-4.000000
41,IL.16,0,4,0,-4.000000
43,IL.6,0,4,0,-4.000000
40,Ferritin,0,3,0,-3.000000
39,MIP.1B,0,3,0,-3.000000
44,IL.8,0,2,0,-2.000000
25,MCP.1.CCL2,1,8,0,-0.777778
36,IL.10,1,5,0,-0.666667


In [ ]:
labels = list(node_info_sort["Feature"].values);
deepPINK_p_values = list(node_info_sort["D_P"].values);
causalMGM_p_values = list(node_info_sort["C_P"].values);
intersection_p_values = list(node_info_sort["I_P"].values);

plt.figure(figsize=(14,8));

ind = np.arange(node_info_sort.shape[0]);
width = 0.7;

p1 = plt.bar(ind, causalMGM_p_values, width,color="blue");
p2 = plt.bar(ind, intersection_p_values, width, bottom=causalMGM_p_values,color="purple");

bottom_values = [];
for index in range(0,len(intersection_p_values)):
    temp_val = causalMGM_p_values[index]+intersection_p_values[index];
    bottom_values.append(temp_val);
# print(causalMGM_p_values)
# print(intersection_p_values)
# print(bottom_values)
p3 = plt.bar(ind, deepPINK_p_values, width, bottom=bottom_values,color="red")

plt.ylabel('# of associations',fontsize=18);
plt.xticks(ind, labels,rotation=90,fontsize=18);
plt.yticks(fontsize=18);
plt.legend((p1[0], p2[0],p3[0]), ('CausalMGM', "Overlapped",'DeepPINK'));
plt.xlim([-1,ind.size])

# plt.savefig("Numbers of assocs uniquely in DeepPINK and uniquely in causalMGM and in both for each feature(sorted_by_proportion)1.svg")

In [ ]:
# associations = pd.read_csv('ML & Importance for DeepPINK&CausalMGM associations_v3.csv',index_col=0);
# nonlinearList = [];
# linearList = [];
# for index,row in associations.iterrows():
#     strs = index.split('_');
#     f1 = strs[0];
#     f2 = strs[0];
#     if row['DeePPINK']=='yes':
#         nonlinearList.append({'F1':f1,'F2':f2});
#     if row['CausalMGM']=='yes':
#         linearList.append({'F1':f1,'F2':f2});
# pd.DataFrame(nonlinearList).to_csv('Supplementary Data Nonlinear.csv')
# pd.DataFrame(linearList).to_csv('Supplementary Data Linear.csv')